# 🤖 OpenVLA LIBERO Evaluation on Google Colab

이 노트북은 Google Colab에서 OpenVLA의 LIBERO Spatial evaluation을 처음부터 끝까지 실행하는 방법을 보여줍니다.

This notebook shows how to run OpenVLA's LIBERO Spatial evaluation on Google Colab from start to finish.

---

## ⚙️ 사전 준비 / Prerequisites

1. **GPU 활성화**: Runtime → Change runtime type → GPU (T4 추천)
2. **Enable GPU**: Runtime → Change runtime type → GPU (T4 recommended)

---

## 📋 Step 1: GPU 확인 / Check GPU

먼저 GPU가 제대로 할당되었는지 확인합니다.

In [ ]:
!nvidia-smi

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU detected: {gpu_name}")
    print(f"✅ Total GPU memory: {gpu_memory:.2f} GB")
    
    if "T4" in gpu_name:
        print("\n⚠️  T4 GPU detected - You should use --load_in_8bit True")
    elif "V100" in gpu_name or "A100" in gpu_name:
        print("\n✅ High-end GPU - You can run without quantization")
else:
    print("❌ NO GPU DETECTED!")
    print("Please enable GPU: Runtime > Change runtime type > GPU")

## 📦 Step 2: 저장소 클론 / Clone Repository

**중요!** 원본 OpenVLA가 아니라 **trillion-boy/openvla** fork를 클론하고,
**claude/libero-spatial-eval-setup-Xhupi** 브랜치를 체크아웃합니다.

**Important!** Clone the **trillion-boy/openvla** fork (not the original),
and checkout the **claude/libero-spatial-eval-setup-Xhupi** branch.

In [ ]:
# 저장소 클론 (이미 존재하면 스킵)
import os

if not os.path.exists('/content/openvla'):
    print("[*] Cloning trillion-boy/openvla repository...")
    !git clone https://github.com/trillion-boy/openvla.git
    print("[✓] Repository cloned!")
else:
    print("[✓] Repository already exists")

# 작업 디렉토리 변경
%cd /content/openvla

# Colab 최적화 브랜치로 체크아웃
print("\n[*] Checking out claude/libero-spatial-eval-setup-Xhupi branch...")
!git fetch origin
!git checkout claude/libero-spatial-eval-setup-Xhupi
print("[✓] Branch checked out!")

# 현재 브랜치 확인
!git branch --show-current

## 🔧 Step 3: 환경 설정 / Setup Environment

Colab 전용 설정 스크립트를 실행합니다. 이 스크립트는:
- GPU 타입 자동 감지
- 올바른 dependency 버전 설치
- Flash Attention 설치 시도 (실패해도 OK)
- LIBERO 및 필수 패키지 설치

Run the Colab-specific setup script. This script:
- Auto-detects GPU type
- Installs correct dependency versions
- Tries to install Flash Attention (OK if it fails)
- Installs LIBERO and required packages

In [ ]:
# Colab 최적화 설정 스크립트 실행
!python experiments/robot/libero/colab_setup_libero.py

## 🔄 Step 4: Colab 최적화 유틸리티 활성화 / Enable Colab-Optimized Utils

SDPA fallback이 포함된 Colab 최적화 버전을 사용합니다.
이렇게 하면 텐서 크기 버그 (291 vs 290)를 피할 수 있습니다!

Use the Colab-optimized version with SDPA fallback.
This avoids the tensor size bug (291 vs 290)!

In [ ]:
# Colab 최적화 유틸리티로 교체
!cp experiments/robot/openvla_utils_colab.py experiments/robot/openvla_utils.py
print("[✓] Colab-optimized utilities enabled!")
print("[✓] Will automatically use SDPA (avoids tensor size bugs)")

## ⚠️ IMPORTANT: 런타임 재시작 / Restart Runtime

**새로운 패키지를 설치했으므로 런타임을 재시작해야 합니다!**

**You must restart the runtime after installing new packages!**

1. Runtime → Restart runtime
2. 아래의 "Step 6"부터 다시 실행 / Re-run from "Step 6" below

---

## 🚀 Step 6: LIBERO Evaluation 실행 / Run LIBERO Evaluation

런타임 재시작 후 여기서부터 실행하세요!

After restarting runtime, start from here!

In [ ]:
# 작업 디렉토리로 이동
%cd /content/openvla

# 현재 브랜치 확인
!git branch --show-current

### 실행 옵션 선택 / Choose Execution Option

GPU에 따라 아래 옵션 중 하나를 선택하세요:

Choose one of the options below based on your GPU:

In [ ]:
# Option A: T4 GPU (8비트 양자화 사용 - 권장)
# Option A: T4 GPU (with 8-bit quantization - recommended)

!python experiments/robot/libero/run_libero_eval.py \
  --model_family openvla \
  --pretrained_checkpoint openvla/openvla-7b-finetuned-libero-spatial \
  --task_suite_name libero_spatial \
  --center_crop True \
  --load_in_8bit True \
  --num_trials_per_task 10

In [ ]:
# Option B: V100/A100 GPU (양자화 없이 - 더 빠름)
# Option B: V100/A100 GPU (without quantization - faster)

!python experiments/robot/libero/run_libero_eval.py \
  --model_family openvla \
  --pretrained_checkpoint openvla/openvla-7b-finetuned-libero-spatial \
  --task_suite_name libero_spatial \
  --center_crop True \
  --num_trials_per_task 10

## 📊 Step 7: 결과 확인 / Check Results

In [ ]:
# 최신 로그 파일 확인
!ls -lt experiments/logs/*.txt | head -5

print("\n" + "="*80)
print("Latest evaluation results:")
print("="*80)

# 최신 로그 파일의 마지막 30줄 출력
!tail -30 $(ls -t experiments/logs/*.txt | head -1)

## 🎥 Step 8: 롤아웃 비디오 확인 / Check Rollout Videos

생성된 비디오를 확인할 수 있습니다.

In [ ]:
# 생성된 비디오 목록
import os
from datetime import datetime

# 오늘 날짜로 비디오 디렉토리 찾기
today = datetime.now().strftime("%Y_%m_%d")
video_dir = f"./rollouts/{today}"

if os.path.exists(video_dir):
    videos = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])
    print(f"Found {len(videos)} videos in {video_dir}:")
    for i, video in enumerate(videos[:10], 1):  # 처음 10개만 표시
        success = "✓" if "success=True" in video else "✗"
        print(f"{i}. {success} {video}")
else:
    print(f"No videos found in {video_dir}")

In [ ]:
# 첫 번째 비디오 재생
from IPython.display import Video
import os
from datetime import datetime

today = datetime.now().strftime("%Y_%m_%d")
video_dir = f"./rollouts/{today}"

if os.path.exists(video_dir):
    videos = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])
    if videos:
        first_video = os.path.join(video_dir, videos[0])
        print(f"Playing: {videos[0]}")
        display(Video(first_video, width=640, height=480))
    else:
        print("No videos found")
else:
    print(f"Directory {video_dir} does not exist")

## 🔧 Troubleshooting

### 텐서 크기 불일치 오류 (291 vs 290)

```
Caught exception: The size of tensor a (291) must match the size of tensor b (290)
```

**해결책**: Step 4에서 `openvla_utils_colab.py`로 교체했는지 확인하세요.

### CUDA Out of Memory

```
torch.cuda.OutOfMemoryError: CUDA out of memory
```

**해결책**: T4 GPU에서는 `--load_in_8bit True` 옵션을 반드시 사용하세요.

### 8비트 양자화 오류

```
RuntimeError: CUDA error: no kernel image is available
```

**해결책**:
```python
!pip uninstall -y bitsandbytes transformers
!pip install bitsandbytes>=0.43.0
!pip install transformers==4.40.1
# 런타임 재시작 후 Step 6부터 다시 실행
```

### 더 많은 도움말

자세한 문제 해결 가이드는 `COLAB_LIBERO_GUIDE.md`를 참조하세요.

For detailed troubleshooting, see `COLAB_LIBERO_GUIDE.md`

---

## 🎯 다른 Task Suites 실행 / Run Other Task Suites

다른 LIBERO task suites도 실행해볼 수 있습니다:

In [ ]:
# LIBERO-Object
!python experiments/robot/libero/run_libero_eval.py \
  --model_family openvla \
  --pretrained_checkpoint openvla/openvla-7b-finetuned-libero-object \
  --task_suite_name libero_object \
  --center_crop True \
  --load_in_8bit True \
  --num_trials_per_task 10

In [ ]:
# LIBERO-Goal
!python experiments/robot/libero/run_libero_eval.py \
  --model_family openvla \
  --pretrained_checkpoint openvla/openvla-7b-finetuned-libero-goal \
  --task_suite_name libero_goal \
  --center_crop True \
  --load_in_8bit True \
  --num_trials_per_task 10

In [ ]:
# LIBERO-10 (Long Horizon)
!python experiments/robot/libero/run_libero_eval.py \
  --model_family openvla \
  --pretrained_checkpoint openvla/openvla-7b-finetuned-libero-10 \
  --task_suite_name libero_10 \
  --center_crop True \
  --load_in_8bit True \
  --num_trials_per_task 10

## 💾 결과 저장 / Save Results (Optional)

Google Drive에 마운트하여 결과를 저장할 수 있습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 로그 복사
!mkdir -p /content/drive/MyDrive/openvla_results
!cp -r experiments/logs/* /content/drive/MyDrive/openvla_results/
!cp -r rollouts/* /content/drive/MyDrive/openvla_results/

print("[✓] Results saved to Google Drive: MyDrive/openvla_results/")

---

## 📚 참고 자료 / References

- [OpenVLA Paper](https://arxiv.org/abs/2406.09246)
- [LIBERO Project](https://libero-project.github.io/)
- [Detailed Colab Guide](COLAB_LIBERO_GUIDE.md)
- [GitHub Repository](https://github.com/trillion-boy/openvla)

---

**Happy Evaluating! 🚀**